# 🦌 Compounds 🧪

Experiments on building graph fragments that can be composed. This is tricky because to
make reusable fragments a new `id` will have to be stamped out for each element. This
notebook introduces the following elements in `ipyelk.elements`:

- `Node` - wrapper for `ElkNode`
- `Port` - wrapper for `ElkPort`
- `Label` - wrapper for `ElkLabel`
- `Edge` - wrapper for `ElkEdge`
- `Partition` - extends node and has some convience functions for building edges

To stamp out `id`s while remembering the originating objects, a `MarkFactory` class can
be instantiated that owns the `Registry` to maintain the mapping.

In [ ]:
import importnb
import ipywidgets as W
from IPython.display import display

import ipyelk.nx
import ipyelk.tools
import ipyelk.tools.tools
from ipyelk import Elk
from ipyelk.diagram import elk_model
from ipyelk.diagram import layout_options as opt
from ipyelk.elements import Edge, Label, MarkFactory, Node, Port

`ipyelk.contrib.library.activity` extends the base `Elements` into a set of marks that
are appropriate for creating Activity Diagrams. These new marks do not have behaviors or
rules that enforce for how they can be connected.

In [ ]:
from ipyelk.contrib.library.activity import (
    Activity,
    ActivityDiagram,
    Decision,
    EndActivity,
    Join,
    Merge,
    StartActivity,
)


def activity_app():
    """Utility function for creating a new Elk app suitable for an Activity Diagram"""
    diagram_opts = opt.OptionsWidget(
        options=[opt.Direction(value="DOWN"), opt.HierarchyHandling()]
    ).value

    # configure app
    app = Elk(
        transformer=ipyelk.nx.XELK(
            layouts={
                elk_model.ElkRoot: {
                    "parents": diagram_opts,
                },
            },
        ),
        layout={"height": "100%"},
    )
    toggle = ipyelk.tools.tools.ToggleCollapsedBtn(app=app)
    fit = ipyelk.tools.tools.FitBtn(app=app)
    app.toolbar.commands = [fit, toggle]
    return app

## Example Email Activities

Simple representation of processing an email inbox.

In [ ]:
def email_activity_example():

    # Building Elements
    act = ActivityDiagram()

    start = StartActivity()
    end = EndActivity()

    open_email = Activity.make("open email")
    delete_email = Activity.make("delete email")
    read_email = Activity.make("read email")
    reply_email = Activity.make("reply")

    j1 = Join()

    m1 = Merge()

    triage = Decision()
    triage.true.labels = [Label(text="is important")]
    triage.false.labels = [Label(text="is junk")]

    response = Decision()
    response.true.labels = [Label(text="yes")]
    response.false.labels = [Label(text="no")]

    # Connect Elements
    act[start:open_email]
    act[open_email : triage.input : "opening"]
    act[triage.false : delete_email]
    act[delete_email:m1]
    act[triage.true : read_email]
    act[read_email : response.input]
    act[response.false : m1]
    act[response.true : reply_email]
    act[reply_email:m1]
    act[m1:end]

    # build app
    ilk = MarkFactory()
    app = activity_app()
    app.transformer.source = ilk(act)
    app.diagram.symbols = act.symbols
    app.style = act.style
    return app, act

In [ ]:
if __name__ == "__main__":
    email_act_app, email_activities = email_activity_example()
    display(email_act_app)

## Example Email Activities

Simple representation of processing an email inbox.

In [ ]:
def website_activity_example():
    priority_edge_opts = {
        "org.eclipse.elk.layered.priority.direction": "10",
    }

    # Building Elements
    act = ActivityDiagram()

    start = StartActivity()
    end = EndActivity()

    landing = Activity.make("Landing Page")
    login = Activity.make("Login", container=True)
    enter_creds = Activity.make("Enter Credentials")
    register = Activity.make("Register", container=True)
    registration = Activity.make("Enter Registration Data")
    confirm_email = Activity.make("Receive Confirmation Email")
    confirm = Activity.make("Click Confirmation Link")

    website = Activity.make("Explore Website")

    login.add_child(enter_creds, "cred")
    register.add_child(registration, "registration")
    register.add_child(confirm_email, "confirm_email")
    register.add_child(confirm, "confirm")

    d1 = Decision()
    d1.true.labels = [Label(text="registered")]
    d1.false.labels = [Label(text="not registered")]

    d2 = Decision()
    d2.true.labels = [Label(text="logged in")]
    d2.false.labels = [Label(text="not logged in")]

    response = Decision()
    response.true.labels = [Label(text="yes")]
    response.false.labels = [Label(text="no")]

    # Connecting Elements
    act[start:landing].layoutOptions.update(priority_edge_opts)
    act[landing : d1.input].layoutOptions.update(priority_edge_opts)
    act[d1.true : enter_creds]
    act[d1.false : registration]
    act[registration:confirm_email]
    act[confirm_email:confirm]

    m1 = Merge()
    act[enter_creds:m1]
    act[confirm:m1]
    act[m1 : d2.input]
    act[d2.false : landing]
    act[d2.true : website]

    act[website:end]

    # Creating App and setting the source
    ilk = MarkFactory()
    app = activity_app()
    # need to include the other roots `register` and `login` that are not directly connected in `act`
    app.transformer.source = ilk(act, register, login)
    app.diagram.symbols = act.symbols
    app.style = act.style
    return app

In [ ]:
if __name__ == "__main__":
    website_app = website_activity_example()
    display(website_app)

# Record Nodes

Example showing the combination of blocks and activities

In [ ]:
import importnb
import ipywidgets as W
import traitlets as T
from IPython.display import display

import ipyelk.nx
import ipyelk.tools
import ipyelk.tools.tools
from ipyelk import Elk
from ipyelk.contrib.library.block import Aggregation, Block, BlockDiagram, Composition
from ipyelk.diagram import elk_model
from ipyelk.diagram import layout_options as opt
from ipyelk.elements import (
    Compartment,
    Edge,
    Label,
    Mark,
    MarkFactory,
    Node,
    Port,
    Record,
)


class ToggleRecordBtn(ipyelk.tools.tools.ToggleCollapsedBtn):
    def get_related(self, node):
        tree = self.app.transformer.source[1]
        if isinstance(node, Mark) and isinstance(node.element, Compartment):
            parent = list(tree.predecessors(node))[0]
            return [child for i, child in enumerate(tree.neighbors(parent)) if i > 0]
        return super().get_related(node)


def block_app():
    """Utility function for creating a new Elk app suitable for an Activity Diagram"""
    diagram_opts = opt.OptionsWidget(
        options=[opt.Direction(value="RIGHT"), opt.HierarchyHandling()]
    ).value

    # configure app
    app = Elk(
        transformer=ipyelk.nx.XELK(
            layouts={
                elk_model.ElkRoot: {
                    "parents": diagram_opts,
                },
            },
        ),
        layout={"height": "100%"},
    )
    toggle = ToggleRecordBtn(app=app)
    fit = ipyelk.tools.tools.FitBtn(app=app)
    app.toolbar.commands = [fit, toggle]
    return app


def car_example():
    bd = BlockDiagram()

    # Nodes
    vehicle = Block(width=220)
    vehicle.title = Compartment(headings=["Vehicle", "«block»"])
    vehicle.behaviors = Compartment(headings=["Behavior"], content=[" "])

    wheel = Block(width=180)
    wheel.title = Compartment(headings=["Wheel", "«block»"])
    wheel.attrs = Compartment(headings=["properties"], content=["- radius: float"])

    wheel_break = Block()
    wheel_break.title = Compartment(headings=["Break", "«block»"])
    tire = Block()
    tire.title = Compartment(headings=["Tire", "«block»"])

    engine = Block(width=180)
    engine.title = Compartment(headings=["Engine", "«block»"])

    # Edges
    bd[vehicle:wheel:Composition]
    bd[vehicle:engine:Composition]
    bd[wheel:wheel_break:Composition]
    bd[wheel:tire:Composition]

    # internal activities of car
    act = ActivityDiagram().add_class("internal")
    act.start = Activity.make("start engine")
    act.drive = Activity.make("drive")
    act.park = Activity.make("park")
    act[act.start : act.drive]
    act[act.drive : act.park]

    behavior = vehicle.behaviors.add_child(act, "activites")

    # merge defs for both block and activities
    bd.symbols = bd.symbols.merge(act.symbols)
    return bd


def example_car_blocks():
    car = car_example()
    app = block_app()
    cp = MarkFactory()
    app.transformer.source = cp(car)
    app.style = car.style
    app.diagram.symbols = car.symbols

    return app

In [ ]:
if __name__ == "__main__":
    car_app = example_car_blocks()
    display(car_app)